In [41]:
import pandas as pd
import random
import time
from datetime import date

import mysql.connector as mysql
import datetime as dt
import matplotlib.pyplot as plt
from pandasql import sqldf

In [58]:
my_connection = mysql.connect(
host="giniewicz.it",
user="student",
password="57u>3n7"
)

In [151]:
my_connection = mysql.connect(
host="giniewicz.it",
user="team9",
password="te@mqP@ss"
)

In [152]:
cursor = my_connection.cursor()

In [101]:
def generate_name(file):
    """
    Funkcja zwraca wylosowane imię ze 100 najbardziej znanych w Polsce imion. 
    """
    
    df = pd.read_csv(file)[1:100]
    n = df["IMIĘ_PIERWSZE"].sample(1)
    
    return n.to_string(index=False)

In [4]:
generate_name("IMIONA_ZENSKIE.csv")

'MARTA'

In [5]:
generate_name("IMIONA_MESKIE.csv")

'BARTŁOMIEJ'

In [6]:
def generate_surname(file):
    """
    Funkcja zwraca wylosowane nazwisko ze 100 najbardziej znanych w Polsce nazwisk. 
    """
    
    df = pd.read_csv(file)[1:100]
    n = df["Nazwisko aktualne"].sample(1)
    
    return n.to_string(index=False)

In [7]:
generate_surname("NAZWISKA_ZENSKIE.csv")

'SZCZEPANIAK'

In [8]:
generate_surname("NAZWISKA_MESKIE.csv")

'PIOTROWSKI'

In [9]:
def str_time_prop(start, end, time_format, prop):
    """
    Zwraca datę pomiędzy dwiema ustalonymi (w odstępie takim jak argument (0,1))
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))

In [83]:
def random_date(start, end):
    """
    Losuje odstęp i wywołuje funkcje zwracającą date
    
    W ogólności funkcja za pomocą funkcji str_time_prop zwraca losową datę pomiędzy dwiema zadanymi 
    w argumentach danymi. Będzie do losowania JoinDate
    """
    prop=random.random()
    d = str_time_prop(start, end, '%d/%m/%Y', prop)
    return dt.datetime.strptime(d, '%d/%m/%Y')

In [84]:
random_date("01/01/1922", "01/01/1962")

datetime.datetime(1931, 10, 18, 0, 0)

In [72]:
def generate_email(name, surname):
    
    """
    Funkcja zwraca email osoby 
    """
    
    number = int(random.random()*10000)
    mail = name.lower() + '.' + surname.lower() + str(number) + '@giniewicz.it.com'
    
    return mail

In [13]:
name = generate_name("IMIONA_ZENSKIE.csv")
surname = generate_surname("NAZWISKA_ZENSKIE.csv")
email(name, surname)

'magdalena.ziółkowska2848@giniewicz.it.com'

In [14]:
def generate_phone():
    
    """
    Funkcja zwraca numer osoby 
    """
    
    phone = random.randint(500000000, 900000000)
    return phone

In [15]:
generate_phone()

522679007

In [16]:
df = pd.read_csv("ADRESY_beta.csv", sep = ";")
df = df[["MIEJCOWOSC", "NAZWA_ULICY", "NUMER"]]

In [17]:
df.to_csv('ADRESY.csv', index = False)

In [18]:
def generate_address(file): 
    
    df = pd.read_csv(file)
    row = df.sample(1)
    
    city = row["MIEJCOWOSC"].to_string(index=False)
    street = row["NAZWA_ULICY"].to_string(index=False)
    number = row["NUMER"].to_string(index=False)
    
    address = street + " " + number + " " + city
    
    return address

In [19]:
generate_address("ADRESY.csv")

'ulica Potażowa 31 Warszawa'

In [27]:
def generate_gender():

    x = random.random()
    
    if 0 < x < 0.01:
        return "OTHER"
    elif 0.01 < x < 0.522:
        return "FEMALE"
    else:
        return "MALE"

In [28]:
generate_gender()

'MALE'

In [142]:
def generate_depart_date(company_start, current_date):
    
    """
    Funkcja zwraca datę odejścia pracownika lub gracza,
    musimy ustalić z jakimś prawdopodobieństwem ludzie odchodzą
    """
    
    x = random.random()
    
    if 0 < x < 0.1:
        date = random_date(company_start, current_date)
    else:
        date = "01/01/9999"
        date = dt.datetime.strptime(date, '%d/%m/%Y')
    
    
    return date

In [145]:
generate_depart_date("01/01/1922", "01/06/2022")

datetime.datetime(1979, 2, 18, 0, 0)

In [ ]:
def generate_income(min_income, max_income):
    
    """
    Funckcja zwraca losowe dochody danego pracownika.
    """
    
    income = random.randint(min_income ,max_income)
    
    return income

In [153]:
def fill(cursor):
    
    staff_depart_dates = []
    def info_fill_staff():


        for i in range(15):

            gender = generate_gender()
            if gender == "FEMALE":
                file1 = "IMIONA_ZENSKIE.csv"
                file2 = "NAZWISKA_ZENSKIE.csv"
            elif gender == "MALE":
                file1 = "IMIONA_MESKIE.csv"
                file2 = "NAZWISKA_MESKIE.csv"
            else:
                x = random.random()
                if 0 < x < 0.5:
                    file1 = "IMIONA_ZENSKIE.csv"
                    file2 = "NAZWISKA_ZENSKIE.csv"
                else:
                    file1 = "IMIONA_MESKIE.csv"
                    file2 = "NAZWISKA_MESKIE.csv"


            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            join_date = dt.date(2021, 1, 1)
            depart_date = generate_depart_date("01/01/2021", "01/06/2022")

            if depart_date != dt.datetime(9999, 1, 1, 0, 0):
                staff_depart_dates.append((i, depart_date))

            birth_date = random_date("01/01/1958", "01/01/2004")
            email = generate_email(first_name, last_name)
            phone = generate_phone()

            qr = ('INSERT INTO team9.Info ' 
                  '(InfoID, FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (i+1, first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
            
            
            
        for i in range(len(staff_depart_dates)):
            
            gender = generate_gender()
            if gender == "FEMALE":
                file1 = "IMIONA_ZENSKIE.csv"
                file2 = "NAZWISKA_ZENSKIE.csv"
            elif gender == "MALE":
                file1 = "IMIONA_MESKIE.csv"
                file2 = "NAZWISKA_MESKIE.csv"
            else:
                x = random.random()
                if 0 < x < 0.5:
                    file1 = "IMIONA_ZENSKIE.csv"
                    file2 = "NAZWISKA_ZENSKIE.csv"
                else:
                    file1 = "IMIONA_MESKIE.csv"
                    file2 = "NAZWISKA_MESKIE.csv"


            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            birth_date = random_date("01/01/1958", "01/01/2004")
            email = generate_email(first_name, last_name)
            phone = generate_phone()
            join_date = staff_depart_dates[i][1]
            depart_date = dt.datetime(9999, 1, 1, 0, 0)
            
            qr = ('INSERT INTO team9.Info ' 
                  '(InfoID, FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (i+16, first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))  
            
    info_fill_staff()


In [154]:
fill(cursor)

In [71]:
fill_info("Tymek",  "Kempa", "Warszawa", dt.date(1977, 6, 14), dt.date(1977, 6, 14), dt.date(1977, 6, 14), "giniewicz.it", "567567567", "MALE", cursor)

In [60]:
qr = 'SELECT * FROM sakila.actor LIMIT 10'
cursor.execute(qr)
cursor.fetchall()

[(1, 'PENELOPE', 'GUINESS', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (2, 'NICK', 'WAHLBERG', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (3, 'ED', 'CHASE', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (4, 'JENNIFER', 'DAVIS', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (5, 'JOHNNY', 'LOLLOBRIGIDA', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (6, 'BETTE', 'NICHOLSON', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (7, 'GRACE', 'MOSTEL', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (8, 'MATTHEW', 'JOHANSSON', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (9, 'JOE', 'SWANK', datetime.datetime(2006, 2, 15, 4, 34, 33)),
 (10, 'CHRISTIAN', 'GABLE', datetime.datetime(2006, 2, 15, 4, 34, 33))]

In [155]:
my_connection.commit()
cursor.close()
my_connection.close()